In [2]:
import os
import pandas as pd
import pandas_datareader as dr
import numpy as np
from tqdm import tqdm
import glob

import time
import math
import warnings
from datetime import datetime,timedelta,date
#import pandas_market_calendars as mcal
import dtale
import QuantLib as ql

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

from ib_insync.contract import * 
from ib_insync import *
import utils
from fpdf import FPDF

warnings.filterwarnings('ignore')
pd.set_option('max.rows',5000)
pd.set_option('max.columns',150)

# tlg = '/Users/david/Downloads/U3913547_20200424_20210310.tlg'
tlg = '/Users/david/Downloads/trade_log.tlg'
# tlg = '/Users/david/Downloads/osap_account.tlg'
# tlg = '/Users/david/Downloads/U3913547_20200424_20210325.tlg'
# ofx_file = '/Users/david/Downloads/U3913547_20210101_20210304.qfx'

/usr/local/lib/python3.9/site-packages/dash_bootstrap_components/_table.py:5: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [12]:
c

In [27]:
df = yh.Ticker(['AAPL','NFLX'],asynchronous=True).history(start='2020-01-01', interval='1h')

In [29]:
df.rename(columns={'date':'date_time'})

high        open       close         low  \
symbol date                                                                  
AAPL   2020-01-02 10:30:00   74.602501   74.059998   74.385002   73.797501   
       2020-01-02 11:30:00   74.687500   74.385002   74.527496   74.277496   
       2020-01-02 12:30:00   74.699997   74.525002   74.571251   74.362503   
       2020-01-02 13:30:00   74.688400   74.572502   74.623474   74.572502   
       2020-01-02 14:30:00   74.690002   74.625000   74.672501   74.514999   
...                                ...         ...         ...         ...   
NFLX   2021-10-11 11:30:00  637.579285  636.659973  635.609985  634.450012   
       2021-10-11 12:30:00  636.869995  635.440002  631.451111  630.859985   
       2021-10-11 13:30:00  632.729980  631.364990  631.630188  630.919983   
       2021-10-11 14:30:00  632.580017  631.770020  631.619995  630.760010   
       2021-10-11 15:30:00  631.989990  631.630005  626.820007  626.780029   

                               volume  dividends  splits  
symbol date                                               
AAPL   2020-01-02 10:30:00  9941640.0        0.0     0.0  
       2020-01-02 11:30:00  4795404.0        0.0     0.0  
       2020-01-02 12:30:00  3345181.0        0.0     0.0  
       2020-01-02 13:30:00  1936326.0        0.0     0.0  
       2020-01-02 14:30:00  2432900.0        0.0     0.0  
...                               ...        ...     ...  
NFLX   2021-10-11 11:30:00   195995.0        0.0     0.0  
       2021-10-11 12:30:00   294656.0        0.0     0.0  
       2021-10-11 13:30:00   209804.0        0.0     0.0  
       2021-10-11 14:30:00   271942.0        0.0     0.0  
       2021-10-11 15:30:00   471184.0        0.0     0.0  

[6259 rows x 7 columns]

In [12]:
#Start connection with TWS or IB Gateway open
util.startLoop()  # uncomment this line when in a notebook

ib = IB()
try:
    #TWS Connection Live
    ib.connect('127.0.0.1', 7497, clientId=1)
    #ib.setCallback('error', onError)
    print('Connection Accepted through TWS')
except:
    #IB Gateway connection
    ib.connect('127.0.0.1', 4001, clientId=1)
    print('Connection Accepted through Gateway')
    pass

Connection Accepted through TWS


In [24]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

def ql_to_datetime(d):
    return datetime(d.year(), d.month(), d.dayOfMonth())

def onError(reqId, errorCode, errorString, contract):
    print("ERROR", reqId, errorCode, errorString)
    if errorCode == 200 and errorString == 'No security definition has been found for the request':
        print("Bad contract")
        pass
    elif errorCode == 1102:
        print("Restarting after outage")
        main()

In [25]:
def parse_tlg(file):
    
    #Reads in file and parses to dataframe
    log = pd.read_csv(file,sep='delimiter', header=None,engine='python')
    log = log[0].str.split('|',expand=True)
    log = log.fillna(value=np.nan)
    
    #Variables
    headers = []
    headers_index = sorted([len(log)])
    dfs = {}
    previous_index = 0
    count=0
    
    #Iterates through df rows
    for line in range(0,len(log)):
        
        #Indexes by titles
        #TODO probably a better way to do this than by > 7. Fix later...maybe
        if len(log[0].iloc[line]) > 7:
            
            #Creates header name and index 
            header = log[0].iloc[line]
            header_index = log.index.get_loc(line)
            
            
            #Stores header name and index  to list 
            headers.append(header)
            headers_index.append(header_index)
            
            #appends multiple dataframes for each category header to dict for output
        previous_index = header_index
        headers_index = sorted(headers_index)
        
        
    for header in headers:
        index1 = headers_index[count]
        index2 = headers_index[count+1]
        dfs[header] = log.iloc[index1:index2]
        
        count += 1
        
        
    #Column Transformations to output proper format
    dfs['OPTION_TRANSACTIONS']['symbol'] = dfs['OPTION_TRANSACTIONS'][2].str.split(' ').str[0]
    dfs['OPTION_TRANSACTIONS'].columns = ['option_txs','fitid','option_ticker','opt_description',
                                          'exchange','optselltype','buy_sell_type','date','time',
                                          'currency','units','100','unit_price',
                                          'total_price','commission','currate','symbol'
                                         ]
    
    dfs['OPTION_TRANSACTIONS']['buy_sell_type'] = dfs['OPTION_TRANSACTIONS']['buy_sell_type'].str.split(';').str[0]
    
    dfs['OPTION_TRANSACTIONS']['strike'] = dfs['OPTION_TRANSACTIONS']['opt_description'].str.split(' ').str[2]
    dfs['OPTION_TRANSACTIONS']['opt_type']  = dfs['OPTION_TRANSACTIONS']['opt_description'].str.split(' ').str[-1]
    dfs['OPTION_TRANSACTIONS']['expiration'] = dfs['OPTION_TRANSACTIONS']['option_ticker'].str.split(' ').str[-1].str.split('C').str[0].str.split('P').str[0]
    dfs['OPTION_TRANSACTIONS'] = dfs['OPTION_TRANSACTIONS'].iloc[1:]
    dfs['OPTION_TRANSACTIONS']['date_time'] = dfs['OPTION_TRANSACTIONS']['date']+' '+dfs['OPTION_TRANSACTIONS']['time']
    pd.to_datetime(dfs['OPTION_TRANSACTIONS']['date_time'],format='%Y%m%d %H:%M:%S')
    dfs['OPTION_TRANSACTIONS']['date_time'] = dfs['OPTION_TRANSACTIONS']['date_time'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d %H:%M:%S'))
    
    return dfs

In [26]:
# tt =dr.DataReader("AAPL", "av-intraday", 
#                start='2021-01-01',
#                end='2021-04-01',
#                api_key='9SV5T3LUBDMD729S')

In [27]:
# tt.index = tt.index.astype('datetime64[ns]')

In [28]:
# tt[tt.index.minute ==0]

In [29]:
# import yfinance as yf
# data = yf.download("AAPL", start="2020-01-01", end="2021-01-01",interval='1H')

In [30]:
def position_data (ticker,opt_tx,engine='ibkr'):
    if engine == 'ibkr':
        return get_ibkr_data(ticker,opt_tx)
    elif engine == 'av':
        return
        
        

In [31]:
def get_ibkr_data(ticker,opt_tx):
    '''
    Returns df, txs, ticker
    '''
    
    txs = opt_tx[opt_tx['symbol'] == ticker]
    txs['date'] = pd.to_datetime(txs['date'],format='%Y%m%d')


    #Convert Expiration to DT
    txs['expiration'] = pd.to_datetime(txs['expiration'],yearfirst=True)
    txs['date_time'] = txs['date_time'].apply(lambda x: hour_rounder(pd.to_datetime(x))).astype('datetime64[ns]')
    txs['units'] = txs['units'].astype('float64')

    expir_date = txs['expiration'].max()
    currency = txs['currency'].iloc[0]
    start_date = txs['date'].min()
    days = expir_date - start_date
    today = datetime.today()

    if today > expir_date:
        days = (today-start_date) + timedelta(60)


     ####### Retrieve Data from IBKR ##########   
    contract = Stock(symbol=ticker,exchange='SMART',currency=currency)
    ib.reqContractDetails(contract)

    if days.days < 365:
        dur_str = f"{days.days} D"

    elif days.days >= 365:
        dur_str = f"{math.ceil(days.days/365)} Y"


#     ib.setCallback('error', onError)
    
    bars = ib.reqHistoricalData(contract,durationStr=dur_str,endDateTime=expir_date,
                                    barSizeSetting='1 hour',whatToShow='TRADES',useRTH=False)

    if util.df(bars).empty:
        contract = Stock(symbol=ticker,exchange='NASDAQ',currency=currency)
        bars = ib.reqHistoricalData(contract,durationStr=dur_str,endDateTime=expir_date,
                                    barSizeSetting='1 hour',whatToShow='TRADES',useRTH=False)
        if util.df(bars) == None:
            contract = Stock(symbol=ticker,exchange='NYSE',currency=currency)
            bars = ib.reqHistoricalData(contract,durationStr=dur_str,endDateTime=expir_date,
                                        barSizeSetting='1 hour',whatToShow='TRADES',useRTH=False)
   
    df = util.df(bars)
    df = df.rename(columns={'date':'date_time'})

    txs = pd.merge(txs,df[['date_time','open','close']],on='date_time', how='left')
    df = df[df['date_time'] >=txs['date_time'].min()-timedelta(60)]
        
    return df,txs, ticker 

In [32]:
def graph(df,mask,ticker):
    '''
    Returns Plotly figure if passed a df containing 1 tickers transactions
    '''
    # standardize column names & create buy/sell dataframes
    df.columns = df.columns.str.lower()
    buys = mask[mask['units']>0]
    sells = mask[mask['units']<0]
    
    fig = make_subplots(rows=2, cols=1,specs=[[{"secondary_y": True}],
                                              [{"secondary_y": True}]])
    
    fig.add_trace(go.Candlestick(x=df['date_time'],
               open=df['open'], 
               high=df['high'],
               low=df['low'], 
               close=df['close'],
               yaxis='y2',
               whiskerwidth=.5,
               name='OHLC',
               line=dict(width=1),
               increasing={'line': {'width': 1}},
               decreasing={'line': {'width': 1}})

             ),

    fig.add_trace(go.Scatter(
                    x=sells['date_time'],
                    y=sells['open']*1.02,
                    mode='markers',
                    name='Sell',
                    marker_symbol='triangle-down',
                    yaxis='y2',
                    marker=dict(
                         color='red',
                         size=14,
                         line=dict(
                            color='DarkSlateGrey',
                            width=1)))
                 ),
    
    fig.add_trace(go.Scatter(
                    x=buys['date_time'],
                    y=buys['close']*.98,
                    mode='markers',
                    name='Buy',
                    marker_symbol='triangle-up',
                    yaxis='y2',
                    marker=dict(
                         color='green',
                         size=14,
                         line=dict(
                            color='DarkSlateGrey',
                            width=1)))),


    fig.update_layout(xaxis=dict(tickformat='%b-%y',rangebreaks=[
                                                                 dict(bounds=["sat", "mon"]),
#                                                                  dict(values=holidays_list),
                                                                 dict(bounds=[17, 9], pattern="hour")
                                                                ]),
                      xaxis_rangeslider_visible=True,
                      height=800,width=1400,
                      hoverdistance=0,hovermode='y',title=f"{ticker} - Total PNL: ${-mask['total_price'].astype('float64').sum()}")
    

    return fig

In [34]:
def holidays(dates):

    start = pd.Timestamp(dates[0],tz=None)
    end = pd.Timestamp(dates.iloc[-1],tz=None)

    calendar = ql.UnitedStates()
    holidays = calendar.holidayList(ql.Date(start.day, start.month, start.year),
                         ql.Date(end.day, end.month, end.year))
    holidays = [ql_to_datetime(x) for x in holidays]
    
    holidays_list = []

    for each_date in holidays:
        for i in range(0,24):
            ts = each_date + timedelta(hours=i)
            holidays_list.append(pd.Timestamp(np.datetime64(ts)).strftime('%Y-%m-%d %H:%M:%S'))
    return holidays_list

In [35]:
def datatable(txs):
    '''
    Returns a plotly datatable figure when feed a txs dataframe
    '''
    
    tx_columns = txs[['date','time','opt_description','optselltype',
                      'exchange','unit_price','units','total_price','currency','currate','commission']]
    #Need to multiply by -1 to make total more intuitive
    tx_columns['total_price'] = tx_columns['total_price'].astype('float64')
    tx_columns['total_price'] = tx_columns['total_price']*-1
    tx_columns['date'] = tx_columns['date'].dt.date
    tx_columns = tx_columns.sort_values(['date','time'])

    fig = go.Figure(data=[go.Table(columnwidth = [12,8,13,11,11,9,5,10,8,8,10],
                                   header=dict(values=list(tx_columns.columns)),
                                   cells=dict(values=tx_columns.T,height=50))])
    fig.update_layout(width=1200)

    return fig

In [36]:
def ticker_data(opt_tx):
    '''
    Returns ticker dict and missing tickers list 
    '''
    ticker_dict ={}
    missing_tickers = list()

    for symbol in tqdm(opt_tx['symbol'].unique()): 
        try:
            df, txs, symbol = get_ibkr_data(ticker,opt_tx)
            ticker_dict[ticker] = dict(ohlc=df,txs=txs)

        except:
            pass
            missing_tickers.append(str(ticker))
            
    print(f"Missing tickers include {missing_tickers}")
        
    return ticker_dict, missing_tickers


In [20]:
#Create folder for chart plots in current directory 
def save_plots(ticker_dict):
    '''
    Saves plots to current working directory
    '''

    try:
        os.mkdir('charts')
        os.mkdir('tables')
    except:
        pass
    charts_path = os.getcwd()+"/"+"charts/"
    table_path = os.getcwd()+"/"+"tables/"

    for ticker, v in tqdm(ticker_dict.items()):
        graph(ticker_dict[ticker]['ohlc'],
              ticker_dict[ticker]['txs'],
              ticker).write_image(f"charts/{ticker}.png",format='png')
        datatable(ticker_dict[ticker]['txs']).write_image(f"tables/{ticker}.png",format='png')
    

In [37]:
dfs = parse_tlg(tlg)
opt_tx = dfs['OPTION_TRANSACTIONS']

In [38]:
ticker_dict, missing_tickers = ticker_data(opt_tx)

100%|██████████| 20/20 [00:00<00:00, 190.24it/s]

Missing tickers include ['BBD', 'AA', 'AAPL', 'AMC', 'BB', 'BLDP', 'BLNK', 'BTAQ', 'CHPT', 'DXC', 'GS', 'LRCX', 'OLN', 'OPAD', 'QQQ', 'SCR', 'SPY', 'TDOC', 'UBER', 'VLDR']


In [29]:
save_plots(ticker_dict)

100%|███████████████████████████████████████████| 18/18 [00:09<00:00,  1.96it/s]


In [30]:
class PDF(FPDF):
    def lines(tx):
        self.set_line_width(0.0)
        self.line(5.0,5.0,205.0,5.0) # top one
        self.line(5.0,292.0,205.0,292.0) # bottom one
        self.line(5.0,5.0,5.0,292.0) # left one
        self.line(205.0,5.0,205.0,292.0) # right one
    def add_plot(self,image_path):
        pdf.image(image_path, x=20, y=8, w=170, h = 100)
        pdf.set_font("Arial", size=12)
        pdf.ln(85)  # move 85 down
        pdf.cell(160, 10, ln=1)
    def add_table(self,image_path):
        pdf.image(image_path, x=10, y=100, w=190, h = 80)
        pdf.set_font("Arial", size=12)
        pdf.ln(85)  # move 85 downs
#         pdf.cell(200, 10, ln=1)
    def notes(self):
        self.set_line_width(0.5)
        self.rect(x=10.0, y=200.0, w=190.0,h=90.0)

In [31]:
pdf = PDF(orientation='P', unit='mm', format='A4')
tickers = [ticker for ticker,v in ticker_dict.items()]

for ticker in tqdm(tickers):
    print(ticker)
    pdf.add_page()
    pdf.lines()
    pdf.notes()
    pdf.add_plot(os.getcwd()+'/'+f"charts/{ticker}.png")
    pdf.add_table(os.getcwd()+'/'+f"tables/{ticker}.png")
    
pdf.output('/Users/david/Documents/Updated_Trades_Oct7.pdf','F')

  0%|                                                    | 0/18 [00:00<?, ?it/s]

AA


  6%|██▍                                         | 1/18 [00:03<00:53,  3.17s/it]

AAPL


 11%|████▉                                       | 2/18 [00:06<00:49,  3.09s/it]

...

UBER


 94%|████████████████████████████████████████▌  | 17/18 [00:52<00:03,  3.07s/it]

VLDR


100%|███████████████████████████████████████████| 18/18 [00:55<00:00,  3.11s/it]


''

In [ ]:
pdf_w=210
pdf_h=297

class PDF(FPDF):
    def lines(self):
        self.set_line_width(0.0)
        self.line(5.0,5.0,205.0,5.0) # top one
        self.line(5.0,292.0,205.0,292.0) # bottom one
        self.line(5.0,5.0,5.0,292.0) # left one
        self.line(205.0,5.0,205.0,292.0) # right one
    def notes(self):
        self.set_line_width(0.5)
        self.rect(10.0, 190.0, 190.0,110.0)
    def add_image(self,image_path):
        pdf.image(image_path, x=10, y=8, w=150, h = 100)
        pdf.set_font("Arial", size=12)
        pdf.ln(85)  # move 85 down
        pdf.cell(200, 10, ln=1)
        
pdf = PDF(orientation='P', unit='mm', format='A4')

pdf.add_page()
# pdf.lines()
pdf.notes()
    
pdf.output('/Users/david/Documents/test2.pdf','F')

<H1> Ticker lookup
    

In [ ]:
ticker = 'AAPL'

In [ ]:

mask = opt_tx[opt_tx['symbol'] == ticker]
mask['date'] = pd.to_datetime(mask['date'],format='%Y%m%d')


#Convert Expiration to DT
mask['expiration'] = pd.to_datetime(mask['expiration'],yearfirst=True)
mask['date_time'] = mask['date_time'].apply(lambda x: hour_rounder(pd.to_datetime(x))).astype('datetime64[ns]')
mask['units'] = mask['units'].astype('float64')

expir_date = mask['expiration'].max()
currency = mask['currency'].iloc[0]
start_date = mask['date'].min()
days = expir_date - start_date
today = datetime.today()

if today > expir_date:
    days = (today-start_date) + timedelta(60)
    print('Expired')
    



# # print(expir_date,'\n',f"{currency}\n",f'{start_date}\n',days)
# print(days)


In [ ]:
# Insert symbol and currency
contract = Stock(symbol=ticker,exchange='SMART',currency=currency)
ib.reqContractDetails(contract)

if days.days < 365:
    dur_str = f"{days.days} D"
    
elif days.days >= 365:
    dur_str = f"{math.ceil(days.days/365)} Y"
    

bars = ib.reqHistoricalData(contract,durationStr=dur_str,endDateTime=expir_date,
                            barSizeSetting='1 hour',whatToShow='TRADES',useRTH=False)

df = util.df(bars)
df = df.rename(columns={'date':'date_time'})

mask = pd.merge(mask,df[['date_time','open','close']],on='date_time', how='left')

In [ ]:
plot_ticker(df,mask,ticker)

In [ ]:
plot_positions('AAPL',opt_tx)    

In [ ]:
import mplfinance as mpf
setup = dict(type='ohlc')
# df = df.set_index('date_time')
mpf.plot(df,style='charles',**setup,figscale=2)

<H2> OFX File Parse

In [ ]:
contract = ib.reqContractDetails(Contract(symbol=conId[0],secType='OPT',lastTradeDateOrContractMonth=, strike, right))[0]

In [190]:
from ofxparse import OfxParser
import codecs
with codecs.open('/Users/david/Downloads/U3913547_20210101_20210224.ofx') as fileobj:
    ofx = OfxParser.parse(fileobj)

In [193]:

def get_seclist():
    seclist = []
    for security in ofx.security_list:
        seclist.append([security.ticker, security.uniqueid, security.name])

    seclist = pd.DataFrame(seclist,columns=['ticker','uniqueid','name'])
    seclist['symbol'] = seclist['ticker'].str.extract(r'([A-Z]{4}|[A-Z]{3}|[A-Z]{2}|[A-Z])')

    return seclist

seclist = get_seclist()

In [201]:
ofx.account.

In [204]:

stmts = ofx.statements  # All {``STMTRS``, ``CCSTMTRS``, ``INVSTMTRS``} in the response
txs = stmts[0].transactions  # The relevant ``*TRANLIST``
acct = stmts[0].account  # The relevant ``*ACCTFROM``
balances = stmts[0].balances  # ``INVBAL`` - use ``balance`` for bank statement ``LEDGERBAL``
securities = ofx.securities

AttributeError: 'Ofx' object has no attribute 'statements'

In [225]:
txs = ofx.account.statement.transactions

for tx in txs:
    print([tx.type, tx.tradeDate , tx.settleDate, tx.memo,
          tx.security, tx.income_type,tx.units,
    tx.unit_price, 
    tx.commission,
    tx.fees,
    tx.total,
    tx.tferaction])


['buyopt', datetime.datetime(2021, 1, 13, 16, 17, 7), None, '', '458039900', '', Decimal('1'), Decimal('6.08'), Decimal('0.6838'), Decimal('0'), Decimal('-608.6838'), None]
['buyopt', datetime.datetime(2021, 1, 13, 16, 17, 7), None, '', '458039900', '', Decimal('1'), Decimal('6.06'), Decimal('-0.4162'), Decimal('0'), Decimal('-605.5838'), None]
['buyopt', datetime.datetime(2021, 1, 27, 18, 24, 22), None, '', '465526487', '', Decimal('1'), Decimal('29.43'), Decimal('0.6338'), Decimal('0'), Decimal('-2943.6338'), None]
['buyopt', datetime.datetime(2021, 1, 29, 18, 17, 44), None, '', '465526522', '', Decimal('1'), Decimal('13.68'), Decimal('0.6338'), Decimal('0'), Decimal('-1368.6338'), None]
['buyopt', datetime.datetime(2021, 2, 4, 19, 59, 35), None, '', '445727310', '', Decimal('1'), Decimal('8.8'), Decimal('0.6838'), Decimal('0'), Decimal('-880.6838'), None]
['buyopt', datetime.datetime(2021, 2, 4, 19, 59, 35), None, '', '445727310', '', Decimal('1'), Decimal('8.8'), Decimal('-0.0162')

AttributeError: 'Transaction' object has no attribute 'tradeDate'

In [203]:
columns = ['tx_type','fitid','dttrade',
           'secid_uniqueid','secid_uniqueidtype','units', 
           'unit_price', 'commission', 'total', 'currency_rate',
           'subacctsec', 'subacctfund', 'optselltype'
          ]

options_df = pd.DataFrame.from_records(options,columns=columns)


In [111]:
ofx.find('ofx')

KeyError: 'invstmttrnrs'

In [110]:
invstmtrs_ofx = ofx.findAll('in')

KeyError: 'invstmttrnrs'

In [63]:
options = []
stocks = []

for tx in txs:
    tx_type = str(tx.__repr__).split('(')[0].split('<')[2]
    print(tx_type)
    if tx_type == 'SELLOPT':
        options.append([tx_type,tx.invtran.fitid,tx.invtran.dttrade,
                       tx.secid.uniqueid,tx.secid.uniqueidtype,
                       tx.units, tx.unitprice, tx.commission, tx.total, tx.currency.currate,
                       tx.subacctsec, tx.subacctfund, tx.optselltype])
        
        print(tx_type)
        print(tx.invtran.fitid)
        print(tx.invtran.dttrade)
        print(tx.secid.uniqueid)
        print(tx.secid.uniqueidtype)
        print(tx.secid.uniqueid)
        print(tx.units)
        print(tx.unitprice)
        print(tx.commission)
        print(tx.total)
        print(tx.currency.currate)
        print(tx.currency.cursym)
        print(tx.subacctsec)
        print(tx.subacctfund)
        print(tx.optselltype)
        
        
    elif tx_type == 'BUYOPT':
        options.append([tx_type,tx.invtran.fitid,tx.invtran.dttrade,
                       tx.secid.uniqueid,tx.secid.uniqueidtype,
                       tx.units, tx.unitprice, tx.commission, tx.total, tx.currency.currate,
                       tx.subacctsec, tx.subacctfund, tx.optbuytype])
        '''
        print(tx_type)
        print(tx.invtran.fitid)
        print(tx.invtran.dttrade)
        print(tx.secid.uniqueid)
        print(tx.secid.uniqueidtype)
        print(tx.units)
        print(tx.unitprice)
        print(tx.commission)
        print(tx.total)
        print(tx.currency.currate)
        print(tx.currency.cursym)
        print(tx.subacctsec)
        print(tx.subacctfund)
        print(tx.optbuytype)
        '''
        
        '''
        
    elif tx_type == 'BUYSTOCK':
        print(tx_type)
        print(tx.invtran.fitid)
        print(tx.invtran.dttrade)
        print(tx.secid.uniqueid)
        print(tx.secid.uniqueidtype)
        print(tx.units)
        print(tx.unitprice)
        print(tx.commission)
        print(tx.total)
        print(tx.currency.currate)
        print(tx.currency.cursym)
        print(tx.subacctsec)
        print(tx.subacctfund)

'''

BUYSTOCK
BUYSTOCK
BUYSTOCK
BUYSTOCK
BUYSTOCK
SELLSTOCK
BUYSTOCK
SELLSTOCK
BUYSTOCK
BUYSTOCK
SELLSTOCK
SELLOPT
SELLOPT
20210113U3913547152559538
2021-01-13 16:17:07+00:00
448941676
CONID
448941676
-1
10.51
0.309027
1050.690973
1.2696
USD
CASH
CASH
SELLTOCLOSE
SELLOPT
SELLOPT
20210113U3913547152559567
2021-01-13 16:17:07+00:00
448941676
CONID
448941676
-1
10.53
0.009071
1052.990929
1.2696
USD
CASH
CASH
SELLTOCLOSE
BUYOPT
BUYOPT
BUYOPT
SELLOPT
SELLOPT
20210129U3913547157078333
2021-01-29 18:17:44+00:00
465526487
CONID
465526487
-1
30.85
0.703978
3084.296021
1.2778
USD
CASH
CASH
SELLTOCLOSE
SELLOPT
SELLOPT
20210127U3913547156025661
2021-01-27 18:24:22+00:00
465526522
CONID
465526522
-1
13.08
0.664707
1307.335293
1.2803
USD
CASH
CASH
SELLTOOPEN
BUYOPT
SELLOPT
SELLOPT
20210112U3913547152312195
2021-01-12 18:24:31+00:00
448213948
CONID
448213948
-1
4.65
0.666076
464.333924
1.2713
USD
CASH
CASH
SELLTOCLOSE
BUYOPT
BUYOPT
SELLOPT
SELLOPT
20210204U3913547158702270
2021-02-04 19:59:35+00:00
445727

In [55]:
options_df

,tx_type,fitid,dttrade,secid_uniqueid,secid_uniqueidtype,units,unit_price,commission,total,currency_rate,subacctsec,subacctfund,optselltype
0,SELLOPT,20210113U3913547152559538,2021-01-13 16:17:07+00:00,448941676,CONID,-1,10.51,0.309027,1050.690973,1.2696,CASH,CASH,SELLTOCLOSE
1,SELLOPT,20210113U3913547152559567,2021-01-13 16:17:07+00:00,448941676,CONID,-1,10.53,0.009071,1052.990929,1.2696,CASH,CASH,SELLTOCLOSE
2,BUYOPT,20210113U3913547152559162,2021-01-13 16:17:07+00:00,458039900,CONID,1,6.08,0.6838,-608.6838,1.2696,CASH,CASH,BUYTOCLOSE
3,BUYOPT,20210113U3913547152559247,2021-01-13 16:17:07+00:00,458039900,CONID,1,6.06,-0.4162,-605.5838,1.2696,CASH,CASH,BUYTOCLOSE
4,BUYOPT,20210127U3913547156025658,2021-01-27 18:24:22+00:00,465526487,CONID,1,29.43,0.6338,-2943.6338,1.2803,CASH,CASH,BUYTOOPEN
5,SELLOPT,20210129U3913547157078333,2021-01-29 18:17:44+00:00,465526487,CONID,-1,30.85,0.703978,3084.296021,1.2778,CASH,CASH,SELLTOCLOSE
6,SELLOPT,20210127U3913547156025661,2021-01-27 18:24:22+00:00,465526522,CONID,-1,13.08,0.664707,1307.335293,1.2803,CASH,CASH,SELLTOOPEN
7,BUYOPT,20210129U3913547157078332,2021-01-29 18:17:44+00:00,465526522,CONID,1,13.68,0.6338,-1368.6338,1.2778,CASH,CASH,BUYTOCLOSE
8,SELLOPT,20210112U3913547152312195,2021-01-12 18:24:31+00:00,448213948,CONID,-1,4.65,0.666076,464.333924,1.2713,CASH,CASH,SELLTOCLOSE
9,BUYOPT,20210204U3913547158702279,2021-02-04 19:59:35+00:00,445727310,CONID,1,8.8,0.6838,-880.6838,1.2824,CASH,CASH,BUYTOOPEN


In [56]:
rows = []


for i in range(0,len(options_df)):
    
    conId = options_df['secid_uniqueid'][i]
    
    
    try :
        contract = ib.reqContractDetails(Contract(conId=conId))[0]
        options_df['contract_details'].iloc[i] = ib.reqContractDetails(Contract(conId=conId))[0].contract
        options_df['symbol'].iloc[i] = ib.reqContractDetails(Contract(conId=conId))[0].contract.symbol
    except:
        pass

options_df['contract_details'] = rows

ValueError: Length of values (0) does not match length of index (71)

In [ ]:
options_df

In [ ]:
options_df['symbol'] = options_df['contract_details']['symbol']

In [ ]:
file = '/Users/david/Downloads/tl1.tlg'

#Reads in file and parses to dataframe
log = pd.read_csv(file,sep='delimiter', header=None,engine='python')
log = log[0].str.split('|',expand=True)
log = log.fillna(value=np.nan)

#Variables
headers = []
headers_index = sorted([len(log)])
dfs = {}
previous_index = 0
count=0

In [ ]:
path = r'/Users/david/Downloads/'                     # use your path
all_files = glob.glob(os.path.join(path, "*.tlg"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f,sep='delimiter', header=None,engine='python') for f in all_files)
cdf = pd.concat(df_from_each_file, ignore_index=True)
cdf = cdf[0].str.split('|',expand=True)
cdf = cdf.fillna(value=np.nan)


#Variables
headers = []
headers_index = sorted([len(log)])
dfs = {}
previous_index = 0
count=0

#Iterates through df rows
for line in range(0,len(cdf)):

    #Indexes by titles
    #TODO probably a better way to do this than by > 7. Fix later...maybe
    if len(cdf[0].iloc[line]) > 7:

        #Creates header name and index 
        header = cdf[0].iloc[line]
        header_index = cdf.index.get_loc(line)


        #Stores header name and index  to list 
        headers.append(header)
        headers_index.append(header_index)

        #appends multiple dataframes for each category header to dict for output
    previous_index = header_index
    headers_index = sorted(headers_index)


for header in headers:
    index1 = headers_index[count]
    index2 = headers_index[count+1]
    dfs[header] = cdf.iloc[index1:index2]

    count += 1


#Column Transformations to output proper format
dfs['OPTION_TRANSACTIONS']['symbol'] = dfs['OPTION_TRANSACTIONS'][2].str.split(' ').str[0]
dfs['OPTION_TRANSACTIONS'].columns = ['option_txs','fitid','option_ticker','opt_description',
                                      'exchange','optselltype','buy_sell_type','date','time',
                                      'currency','units','100','unit_price',
                                      'total_price','commission','currate','symbol'
                                     ]

dfs['OPTION_TRANSACTIONS']['buy_sell_type'] = dfs['OPTION_TRANSACTIONS']['buy_sell_type'].str.split(';').str[0]

dfs['OPTION_TRANSACTIONS']['strike'] = dfs['OPTION_TRANSACTIONS']['opt_description'].str.split(' ').str[2]
dfs['OPTION_TRANSACTIONS']['opt_type']  = dfs['OPTION_TRANSACTIONS']['opt_description'].str.split(' ').str[-1]
dfs['OPTION_TRANSACTIONS']['expiration'] = dfs['OPTION_TRANSACTIONS']['option_ticker'].str.split(' ').str[-1].str.split('C').str[0].str.split('P').str[0]
dfs['OPTION_TRANSACTIONS'] = dfs['OPTION_TRANSACTIONS'].iloc[1:]
dfs['OPTION_TRANSACTIONS']['date_time'] = dfs['OPTION_TRANSACTIONS']['date']+' '+dfs['OPTION_TRANSACTIONS']['time']
pd.to_datetime(dfs['OPTION_TRANSACTIONS']['date_time'],format='%Y%m%d %H:%M:%S')
dfs['OPTION_TRANSACTIONS']['date_time'] = dfs['OPTION_TRANSACTIONS']['date_time'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d %H:%M:%S'))


In [ ]:
dfs

{'ACCOUNT_INFORMATION':                     0         1                      2                     3   \
 1779  OPTION_POSITIONS       NaN                    NaN                   NaN   
 1780           OPT_LOT  U3913547  AAPL  220121C00130000  AAPL 21JAN22 130.0 C   
 1781           OPT_LOT  U3913547  AAPL  220121C00160000  AAPL 21JAN22 160.0 C   
 1782           OPT_LOT  U3913547  ASAN  210521C00030000   ASAN 21MAY21 30.0 C   
 1783           OPT_LOT  U3913547  ASAN  210521C00040000   ASAN 21MAY21 40.0 C   
 1784           OPT_LOT  U3913547  DELL  210416C00077500   DELL 16APR21 77.5 C   
 1785           OPT_LOT  U3913547  GDRX  210219C00035000   GDRX 19FEB21 35.0 C   
 1786           OPT_LOT  U3913547  IPO   210416C00075000    IPO 16APR21 75.0 C   
 1787           OPT_LOT  U3913547  SQ    230120C00195000    SQ 20JAN23 195.0 C   
 1788           OPT_LOT  U3913547  SQ    230120C00250000    SQ 20JAN23 250.0 C   
 1789           OPT_LOT  U3913547  TSM   210416C00105000   TSM 16APR21 105.